In [1]:
#from netgen import gui
from ngsolve.webgui import Draw
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001


In [ ]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [ ]:
gfu_h = GridFunction(X)
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu_h.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)
Draw(gfu.components[0],mesh,"gfu",sd=3)


In [ ]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu_h.vec
gfu_h.vec.data += inv_stokes * res
gfu_N = gfu_h
Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)

In [ ]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
Draw(gfu.components[0],mesh,"gfu",sd=3)

In [ ]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()


inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [ ]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mstar.mat.COO()
Mh_star = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
dn = 1./dt
z = 100
t = 0
tend = 0
erg_h = np.zeros([len(gfu_h.vec),int(dn)+1])
conv_h = np.zeros([len(gfu_h.vec),int(dn)])
print(int(dn))
V = np.zeros([len(gfu.vec),z])
line = np.linspace(0,dn-1,z)

In [ ]:
# implicit Euler/explicit Euler splitting method:
V_sb = MultiVector(gfu.vec, 1)
V_sb[0] = gfu.vec
V[:,0] = np.array(V_sb[0])
V[:,0] = V[:,0]/np.linalg.norm(V[:,0])
erg_h[:,0] = gfu_N.vec

tend += 1
o = 1
e = int(line[o])
d = 0
k = 1
#while t < tend-0.5*dt:
for i in range(int(dn)):
    conv.Assemble()
    conv_h[:,i] = conv.vec
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    for r in line:
        if int(r) == i:
            if i > 0:
                print(o,i,"do Orth:")
                with TaskManager():
                    V_sb.AppendOrthogonalize(gfu.vec)
                V[:,o] = np.array(V_sb[o])
                o = o + 1
    erg_h[:,i+1] = gfu.vec
    #d = d + 1
    #k = k + 1
    t = t + dt
    Redraw()
    #Draw (gfu.components[0], mesh, "gfu")

In [ ]:
MN = np.transpose(V).dot(Mh_star.dot(V))
AN = np.transpose(V).dot(Ah.dot(V))
fN = np.transpose(V).dot(Fh)
#conv_N = np.transpose(V).dot(conv_h.dot(V))


In [ ]:
#velocity_N = gfu_N.components[0]

In [ ]:
uN = np.array(gfu_N.vec).dot(V)
Draw(gfu_N.components[0],mesh,"gfu_N",sd=3)

In [ ]:
#inv_stokes_N = np.linalg.inv(AN)
#res_N = -AN.dot(uN)
#uN += inv_stokes_N.dot(res_N)
#print("uN: ",uN)
#gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))
#Draw(gfu_N.components[0],mesh,"gfu",sd=3)
#Draw(gfu_N.components[0]-gfu_h.components[0],mesh,"err",sd=3)

In [ ]:
#print(np.linalg.norm(np.array(gfu_N.components[0].vec)-np.array(gfu_h.components[0].vec)))#,gfu_h.components[0].vec)
#h = np.array(gfu_N.components[0].vec)-np.array(gfu_h.components[0].vec)
#print(np.max(h))

In [ ]:
#conv = LinearForm(X)
#conv += InnerProduct(grad(velocity)*velocity,v)*dx


In [ ]:
gfu_h.vec.FV().NumPy()[:] = erg_h[:,0]
#Draw(gfu_h.components[0],mesh,"gfu_h_conny",sd=3)
#Draw(gfu_N.components[0]-gfu_h.components[0],mesh,"error",sd=3)

In [ ]:
 #Draw(gfu_N.components[0]-gfu_h.components[0],mesh,"error",sd=3)
 
 erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
 erg_N[:,i+1] = gfu_N.vec
 inv_N = np.linalg.inv(MN)
 for i in range(int(dn)):
    #print(i)
    #conv.Assemble()
    #conv_N = np.transpose(V).dot(conv.vec)
    conv_N = np.transpose(V).dot(conv_h[:,i])
    res_N = AN.dot(uN) + conv_N
    uN -= dt * inv_N.dot(res_N)
    gfu_h.vec.FV().NumPy()[:] = erg_h[:,i]
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))
    Redraw()
    erg_N[:,i+1] = gfu_N.vec

In [ ]:
Residual = []
time = []
for i in range(int(dn)):
    Residual.append(np.linalg.norm(erg_h[:,i]-erg_N[:,i]))
    time.append(dt*i)

In [ ]:
import matplotlib.pyplot as plt
plt.title("Total error {} dt {}".format(z,dt))
plt.ylabel("error")
plt.xlabel("time [s]")
plt.grid()
plt.plot(time,Residual)
plt.savefig("Total error {} dt".format(z))

In [ ]:
v = erg_h[0,999]-erg_N[0,999]
print(v.max())

In [ ]:
erg_N

In [ ]:
print(np.linalg.norm(np.array(gfu_N.components[0].vec)-np.array(gfu.components[0].vec)))

In [ ]:
print(InnerProduct(gfu_N.components[0]-gfu.components[0],gfu_N.components[0]-gfu.components[0]))

In [ ]:
print(X.FreeDofs())

In [ ]:
gfu = GridFunction(X)
velocity = gfu.components[0]
#Draw(velocity,mesh,"u",sd=3)
#Draw(gfu.components[1],mesh,"p",sd=3)
from ngsolve.internal import visoptions
visoptions.scalfunction = "u:0"

# parabolic inflow at bc=1:
uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

#Redraw()

In [ ]:
#uN = np.array(gfu.components[0]).dot(V)
#uN = np.transpose(V).dot(gfu.vec)
uN = np.array(gfu.vec).dot(V)

In [ ]:
u_t_N = GridFunction(X)

inv_stokes_N = np.linalg.inv(AN)
#inv_stokes = a.mat.Inverse(X.FreeDofs())

#res = f.vec.CreateVector()
#res.data = f.vec - a.mat*gfu.vec
res_N = AN.dot(uN)
uN += inv_stokes_N.dot(res_N)
u_t_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))

#gfu.vec.data += inv_stokes * res

velocity_N = u_t_N.components[0]
Draw(velocity_N,mesh,"u_N",sd=3)
#Draw(u_t_h.components[0],mesh,"u_start",sd=3)

In [ ]:
print(u_t_h_satrt)